In [6]:
# %load appendpd.py
import pandas as pd 

list = ["C:/Users/12493/Desktop/python/文本/zh_classifier/ci_new.csv","C:/Users/12493/Desktop/python/文本/zh_classifier/news.csv","C:/Users/12493/Desktop/python/文本/zh_classifier/oldliterature.csv",
        "C:/Users/12493/Desktop/python/文本/zh_classifier/paper.csv","C:/Users/12493/Desktop/python/文本/zh_classifier/poem.csv"]
df_from_each_file = (pd.read_csv(f) for f in list)
concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)
print(concatenated_df)

                                                   content  target
0        start 一曲 阳关 情 几许 end start 知君 欲 秦川 去 end start...       1
1        start 回首 扬州 猖狂 十载 依然 一梦 归来 end start 但觉 安仁 愁鬓 ...       1
2        start 欲 清明 烟雨 细 end start 小槛 临窗 点点 残花 坠 end st...       1
3        start 晓雾 散晴 渚 秋色 满湘山 end start 青鞋 黄帽 名士 共 跻攀 e...       1
4        start 碧 醮 江山 鹤唳 晓 云献 画屏 end start 瑶宫 敞 舞金翔 翠 巍...       1
...                                                    ...     ...
1296839  start 高蹈 时 背 幽怀 事 兼 end start 神仙 君可致 江海 我能 淹 s...       2
1296840  start 岁 岁 荷花 苦未 绿萍 盖 水碍 新荷 end start 请君 六月 重 来...       2
1296841  start 涩 舟沙 水岸 棱 棱 岸上 春泥 深 涴 藤 end start 回首 康山怀...       2
1296842  start 一官何 幸得 十载 无媒 独见 遗 end start 今日 腰下组 请君 看取...       2
1296843  start 世故 吾 问 水滨 向来 不合 典班春 end start 一从 去 作 沧浪 ...       2

[1296844 rows x 2 columns]


In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(concatenated_df["content"],concatenated_df["target"],test_size=0.2, random_state=42)

In [3]:
import numpy as np
from keras.utils.np_utils import to_categorical
output_vec_categorical = to_categorical(y_train,num_classes=6)
print(output_vec_categorical)
from sklearn.utils import class_weight
y_ints = [y.argmax() for y in output_vec_categorical]
print(y_ints[:5])
c_w = class_weight.compute_sample_weight('balanced', y_ints)

print(c_w)
# for i in set(y_ints):
#     print(i)
#     cw[i-1] = c_w[i-1]

[[0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0.]]
[4, 5, 2, 3, 3]
[1.31198073 0.53685779 0.83841784 ... 1.31198073 0.53685779 1.31198073]


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from sklearn import metrics
import pickle

def get_metrics(true_labels, predicted_labels):
    print('Accuracy:', np.round(
        metrics.accuracy_score(true_labels,
                               predicted_labels),2))
    print('Precision:', np.round(
        metrics.precision_score(true_labels,
                                predicted_labels,
                                average='weighted'),2))
    print('Recall:', np.round(
        metrics.recall_score(true_labels,
                             predicted_labels,
                             average='weighted'),  2))
    print('F1 Score:', np.round(
        metrics.f1_score(true_labels,
                         predicted_labels,
                         average='weighted'), 2))
    
def train_predict_evaluate_model(classifier,
                                 train_features, train_labels,
                                 test_features, test_labels):
    # build model
#     print(train_features.shape,train_labels.values.reshape(-1,1).shape)
    clf = classifier.fit(train_features, train_labels.values.reshape(-1,1))
    predictions = classifier.predict(test_features)
    # evaluate model prediction performance
    get_metrics(true_labels=test_labels,
                predicted_labels=predictions)
    with open('model.pkl', 'wb') as f:
        pickle.dump(clf, f, pickle.HIGHEST_PROTOCOL)
    return predictions

mnb = MultinomialNB()
vector = CountVectorizer(max_features = 20000 ,min_df=1, ngram_range=(1,1))
vector2= CountVectorizer(max_features = 20000,min_df=1, ngram_range=(1,2))
tv = TfidfVectorizer(max_features = 30000, max_df = 0.997,lowercase = False)
tv_2gram = TfidfVectorizer(max_features=20000, ngram_range=(1,2),lowercase = False)
print("基于基本词袋模型")
bow_train_features = vector.fit_transform(X_train.astype('U').values)
bow_test_features = vector.transform(X_test.astype('U').values)
mnb_bow_predictions = train_predict_evaluate_model(classifier=mnb,
                                                   train_features=bow_train_features,
                                                  train_labels=y_train,
                                                  test_features=bow_test_features,
                                                  test_labels=y_test)
print("基于bigram词袋模型")
bow2_train_features = vector2.fit_transform(X_train.astype('U').values)
bow2_test_features = vector2.transform(X_test.astype('U').values)
mnb_bow2_predictions = train_predict_evaluate_model(classifier=mnb,
                                                   train_features=bow2_train_features,
                                                  train_labels=y_train,
                                                  test_features=bow2_test_features,
                                                  test_labels=y_test)


print("基于tfidf词袋模型")
tv_train_features = tv.fit_transform(X_train.astype('U').values)
tv_test_features =  tv.transform(X_test.astype('U').values)
mnb_tv_predictions = train_predict_evaluate_model(classifier=mnb,
                                                   train_features=tv_train_features,
                                                  train_labels=y_train,
                                                  test_features=tv_test_features,
                                                  test_labels=y_test)

print("基于tfidf_bigram模型")
tv2_train_features = tv_2gram.fit_transform(X_train.astype('U').values)
tv2_test_features = tv_2gram.transform(X_test.astype('U').values)
mnb_bow_predictions = train_predict_evaluate_model(classifier=mnb,
                                                   train_features=tv2_train_features,
                                                  train_labels=y_train,
                                                  test_features=tv2_test_features,
                                                  test_labels=y_test)
with open('vect.pkl', 'wb') as f:
     # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(tv_2gram, f, pickle.HIGHEST_PROTOCOL)

基于tfidf_bigram模型


D:\anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Accuracy: 0.81
Precision: 0.82
Recall: 0.81
F1 Score: 0.81


In [5]:
bow_train_features

NameError: name 'bow_train_features' is not defined

In [ ]:
sentence2pre = input("输入待测句子：")

In [ ]:
sentence2pre